# Analysis Notebook for Template Technology

In [ ]:
import os
import sys

import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb

sys.path.insert(0, os.path.abspath("../../../src"))

import tyche             as ty

## Load data

In [ ]:
designs = ty.Designs(path = '.',
                     name = 'template.xlsx')
investments = ty.Investments(path = '.',
                             name = 'template.xlsx')

### Compile the production and metric functions for each technology in the dataset.

In [ ]:
designs.compile()

## Evaluate the scenarios in the dataset.

In [ ]:
scenario_results = designs.evaluate_scenarios(sample_count=10)

In [ ]:
scenario_results.xs(1, level="Sample", drop_level=False)

In [ ]:
g = sb.violinplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    data=scenario_results.xs(
        ("Metric", "Overall Efficiency"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],
    dodge=False,
    order=[
        "Current State"              ,
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Overall Efficiency")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')

In [ ]:
g = sb.catplot(
    data=scenario_results.xs(
        ("Metric", "Overall Efficiency"),
        level=["Variable", "Index"]
    ).reset_index(
    )[["Technology", "Scenario", "Value"]],    
    x="Scenario",
    y="Value",
    col="Technology",
    kind="violin",    
    order=[
        "Current State"              ,
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Overall Efficiency")
g.set_xticklabels(rotation=30, ha='right', rotation_mode='anchor')

In [ ]:
g = sb.violinplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    data=scenario_results.xs(
        ("Metric", "Labor"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],
    dodge=False,
    order=[
        "Current State"              ,
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Labor per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')

In [ ]:
g = sb.violinplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    data=scenario_results.xs(
        ("Metric", "System Cost"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],
    dodge=False,
    order=[
        "Current State"              ,
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Annualized System Cost per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')

In [ ]:
g = sb.violinplot(
    x="Scenario",
    y="Value",
    hue="Technology",
    data=scenario_results.xs(
        ("Metric", "Environment"),
        level=["Variable", "Index"]
    ).reset_index()[["Technology", "Scenario", "Value"]],
    dodge=False,
    order=[
        "Current State"              ,
        "All Components Slow Progress"      ,
        "All Components Moderate Progress"  ,
        "All Components Fast Progress"      ,
        "Efficiency Slow Progress"    ,
        "Efficiency Moderate Progress",
        "Efficiency Fast Progress"    ,
        "Peripherals Slow Progress"         ,
        "Peripherals Moderate Progress"     ,
        "Peripherals Fast Progress"         ,
    ]
)
g.set(ylabel="Environmental Impact per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor')

## Evaluate the investments in the dataset.

In [ ]:
investment_results = investments.evaluate_investments(designs, sample_count=100)

### Costs of investments - these calculations are not set up correctly

In [ ]:
investment_results.amounts

### Benefits of investments.

In [ ]:
investment_results.metrics.xs(1, level="Sample", drop_level=False)

In [ ]:
investment_results.summary.xs(1, level="Sample", drop_level=False)

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "Overall Efficiency",
        level="Index"
    ).groupby(["Investment", "Sample"]).sum(numeric_only=True).reset_index()[["Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Overall Efficiency")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor');

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "Environment",
        level="Index"
    ).groupby(["Investment", "Sample"]).sum(numeric_only=True).reset_index()[["Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Environmental Impact per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor');

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "Labor",
        level="Index"
    ).groupby(["Investment", "Sample"]).sum(numeric_only=True).reset_index()[["Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Labor per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor');

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "System Cost",
        level="Index"
    ).groupby(["Investment", "Sample"]).sum(numeric_only=True).reset_index()[["Investment", "Value"]],
    order=[
        "Low Budget, All Categories"      ,
        "Low Budget, Components Only"  ,
        "Medium Budget, All Categories"      ,
        "Medium Budget, Components Only"    ,
        "High Budget, All Categories",
        "High Budget, Components Only"
    ]
)
g.set(ylabel="Annualized System Cost per Unit F")
g.set_xticklabels(g.get_xticklabels(), rotation=30, ha='right', rotation_mode='anchor');

## Multi-objective decision analysis.

### Compute costs and metrics for tranches.

Tranches are atomic units for building investment portfolios. Evaluate all of the tranches, so we can assemble them into investments (portfolios).

In [ ]:
tranche_results = investments.evaluate_tranches(designs, sample_count=50)

Display the cost of each tranche.

In [ ]:
tranche_results.amounts

Display the metrics for each tranche.

In [ ]:
tranche_results.summary

Save the results.

In [ ]:
#tranche_results.amounts.to_csv("example-tranche-amounts.csv")
#tranche_results.summary.to_csv("example-tranche-summary.csv")

### Fit a response surface to the results.

The response surface interpolates between the discrete set of cases provided in the expert elicitation. This allows us to study funding levels intermediate between those scenarios.

In [ ]:
evaluator = ty.Evaluator(tranche_results)

Here are the categories of investment and the maximum amount that could be invested in each:

In [ ]:
evaluator.max_amount

Here are the metrics and their units of measure:

In [ ]:
evaluator.units

#### Example interpolation.

Let's evaluate the case where each category is invested in at half of its maximum amount.

In [ ]:
example_investments = evaluator.max_amount / 2
example_investments

In [ ]:
evaluator.evaluate(example_investments)

Let's evaluate the mean instead of outputing the whole distribution.

In [ ]:
evaluator.evaluate_statistic(example_investments, np.mean)

Here is the standard deviation:

In [ ]:
evaluator.evaluate_statistic(example_investments, np.std)

A risk-averse decision maker might be interested in the 10% percentile:

In [ ]:
evaluator.evaluate_statistic(example_investments, lambda x: np.quantile(x, 0.1))

### ε-Constraint multiobjective optimization

In [ ]:
optimizer = ty.EpsilonConstraintOptimizer(evaluator)

In order to meaningfully map the decision space, we need to know the maximum values for each of the metrics.

#### Example optimization.

Limit spending to $3M.

In [ ]:
investment_max = 3e6

Require that the GHG reduction be at least 40 gCO2e/system and that the Labor wages not decrease.

In [ ]:
metric_min = pd.Series([40, 0], name = "Value", index = ["GHG", "Labor"])
metric_min

Compute the ε-constrained maximum for the LCOE.

In [ ]:
optimum = optimizer.opt_slsqp(
    "LCOE"                       ,
    sense = 'max',
    total_amount = investment_max,
    statistic    = np.mean       ,
)
optimum.exit_message

Here are the optimal spending levels:

In [ ]:
np.round(optimum.amounts)

Here are the three metrics at that optimum:

In [ ]:
optimum.metrics

*Thus, by putting all of the investment into Module R&D, we can expected to achieve a mean 3.75 ¢/kWh reduction in LCOE under the GHG and Labor constraints.*

It turns out that there is no solution for these constraints if we evaluate the 10th percentile of the metrics, for a risk-averse decision maker.

In [ ]:
optimum = optimizer.opt_slsqp(
    "LCOE"                       ,
    sense = 'max',
    total_amount = investment_max,
    statistic    = lambda x: np.quantile(x, 0.1),
)
optimum.exit_message

Let's try again, but with a less stringent set of constraints, only constraining GHG somewhat  but not Labor at all.

In [ ]:
optimum = optimizer.opt_slsqp(
    "LCOE"                                                         ,
    sense = 'max',
    total_amount = investment_max                                  ,
    statistic    = lambda x: np.quantile(x, 0.1)                  ,
)
optimum.exit_message

In [ ]:
np.round(optimum.amounts)

In [ ]:
optimum.metrics

## Reproducing GUI optimization for Git issue 99
See: https://github.com/NREL/tyche/issues/99

### Example MILP optimization

This formulation converts the original non-linear optimization into a piecewise linear, mixed-integer optimization, and then attempts to solve the same problem.

~~As written, the MILP formulation is infeasible.~~

In [ ]:
investment_max = 3e6
metric_min = pd.Series([40, 0], name = "Value", index = ["GHG", "Labor"])

optimum = optimizer.opt_milp(
    "LCOE"                       ,
    sense = 'max',
    total_amount = investment_max,
    statistic    = np.mean       ,
)

optimum.exit_message

Let's determine if the problem is truly infeasible or if the solver is failing by examining the underlying data.

In [ ]:
_wide = evaluator.evaluate_corners_wide(np.mean).reset_index()
_wide

Add a column to the above data with the total investment amount.

In [ ]:
_wide['total R&D'] = _wide.loc[:,'BoS R&D'] + _wide.loc[:, 'Inverter R&D'] + _wide.loc[:, 'Module R&D']
_wide

Look for entries that meet the total investment constraint of less than $3 MM.

In [ ]:
_wide.loc[_wide['total R&D'] <= 3e6,:]

~~This entry does not meet the GHG constraint, which is why the MILP formulation is infeasible.~~